## Sources

In [3]:
import pandas as pd
import numpy as np
import csv
import uuid

### SKOS Concept

In [11]:
PATH = "/home/STual/KG-cadastre/gentilly/data/SKOS_SOURCES.csv"

skos_srcs = pd.read_csv(PATH,header=0)
display(skos_srcs)

,skos:Concept,skos:inScheme,skos:broader,rdfs:label,skos:prefLabel@fr,skos:prefLabel@en,skos:altLabel@fr,skos:altLabel@en,skos:definition
0,Plan,SourceList,NaN,Plan,Plan,Map,NaN,NaN,Plan faisant partie de la documentation cadast...
1,TableauAssemblage,SourceList,Plan,Tableau d'assemblage,Tableau d'assemblage,Assembly map of the cadastral index maps,NaN,NaN,Plan réalisé à l'échelle d'une commune représe...
2,PlanParcellaire,SourceList,Plan,Plan parcellaire,Plan parcellaire,Cadastral index map,NaN,NaN,Plan représentant les parcellaires d'une porti...
3,PlanParcellaireMinute,SourceList,PlanParcellaire,Plan parcellaire : minute,Plan parcellaire : minute,Cadastral index map : working version,NaN,NaN,"Version de travail du plan parcellaire, peut c..."
4,PlanParcellaireFinal,SourceList,PlanParcellaire,Plan parcellaire final,Plan parcellaire final,Cadastral index map : final version,NaN,NaN,Version finale d'un plan parcellaire
5,PlanParcellaireIssuDAtlas,SourceList,PlanParcellaireFinal,Plan parcellaire issu d'atlas,Plan parcellaire issu d'atlas,Cadastral index map : final version extract fr...,NaN,NaN,Version finale d'un plan parcellaire extraire ...
6,RegistreCadastral,SourceList,NaN,Registre cadastral,Registre cadastral,Register,NaN,NaN,Registre faisant partie de la documentation ca...
7,MatriceCadastrale,SourceList,RegistreCadastral,Matrice cadastrale,Matrice cadastrale,Mutation register,NaN,NaN,Registre contenant l'ensemble des mutations af...
8,EtatsDeSections,SourceList,RegistreCadastral,Etats de sections,Etats de sections,Initial register,NaN,NaN,Registre d'état de sections décrivant le parce...
9,MatriceCadastrale1807_1822_Bati_NonBati,SourceList,MatriceCadastrale,Matrice de rôles des propriétés bâties et non ...,Matrice de rôles des propriétés bâties et non ...,NaN,Matrice des propriétés bâties et non bâties (1...,NaN,"Matrice cadastrale créée entre 1807 et 1822, d..."


In [12]:
# Print the list of column headers
print(skos_srcs.columns.tolist())

['skos:Concept', 'skos:inScheme', 'skos:broader', 'rdfs:label', 'skos:prefLabel@fr', 'skos:prefLabel@en', 'skos:altLabel@fr', 'skos:altLabel@en', 'skos:definition']


In [15]:
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import SKOS, RDF, OWL

# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/codes/cadastrenap/sourceType/")
g.bind('srctype', baseuri)

# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
src = Namespace("http://data.ign.fr/def/sources#")
g.bind('cad', cad)
g.bind('src', src)

mlclasse= Namespace("http://data.ign.fr/id/codes/cadastrenap/mlClasse/")
g.bind('mlclasse', mlclasse)

# Iterate over each row in the DataFrame
for index, row in skos_srcs.iterrows():
    subject_uri = URIRef(baseuri + f"{str(row['skos:Concept'])}")
    g.add((subject_uri, RDF.type, OWL.NamedIndividual))
    g.add((subject_uri, RDF.type, cad.SourceType))
    g.add((subject_uri, RDF.type, SKOS.Concept))
    g.add((subject_uri, SKOS.inScheme, URIRef(cad + f"{str(row['skos:inScheme'])}")))
    g.add((subject_uri, SKOS.prefLabel, Literal(row["skos:prefLabel@fr"],lang='fr')))

    if row["skos:altLabel@fr"] == row["skos:altLabel@fr"]:
        g.add((subject_uri, SKOS.altLabel, Literal(row["skos:altLabel@fr"],lang='fr')))

    if row["skos:prefLabel@en"] == row["skos:prefLabel@en"]:
        g.add((subject_uri, SKOS.prefLabel, Literal(row["skos:prefLabel@en"],lang='en')))

    if row["skos:altLabel@en"] == row["skos:altLabel@en"]:
        g.add((subject_uri, SKOS.altLabel, Literal(row["skos:altLabel@en"],lang='en')))

    if row["skos:broader"] == row["skos:broader"]:
        g.add((subject_uri, SKOS.broader, URIRef(baseuri + f"{str(row['skos:broader'])}")))

    if row["skos:definition"] == row["skos:definition"]:
        g.add((subject_uri, SKOS.definition, Literal(row["skos:definition"],lang='fr')))

# Print out the graph in Turtle syntax
print(g.serialize(format='turtle'))

@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix srctype: <http://data.ign.fr/id/codes/cadastrenap/sourceType/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

srctype:CompteFoncier a cad:SourceType,
        owl:NamedIndividual,
        skos:Concept ;
    skos:broader srctype:ZoneDePage ;
    skos:definition "Ensemble de lignes et de cellules qui décrivent les parcelles d'un ou plusieurs propriétaires conjoints ou sucessifs"@fr ;
    skos:inScheme cad:SourceList ;
    skos:prefLabel "Zone située dans une page de matrice correspondant à un compte foncier"@fr .

srctype:EtatsDeSections_Ap_1822 a cad:SourceType,
        owl:NamedIndividual,
        skos:Concept ;
    skos:broader srctype:EtatsDeSections ;
    skos:definition "Registre d'états de sections créé après 1822."@fr ;
    skos:inScheme cad:SourceList ;
    skos:prefLabel "Etats de sections postérieurs à 1822"@fr .

### Instances

In [7]:
PATH = "/home/STual/KG-cadastre/gentilly/data/SOURCES.csv"

sources = pd.read_csv(PATH,header=0)
display(sources)

,CODE_ARCHIVES,rdf:type,rico:name,rico:location,rico:identifier,rsfs:comment/Folios,hasSourceType,hasCreationDate,hasValidTime/hasBeginning,hasValidTime/hasEnd,rico:isOrWasInstantiationOf,rico:isOrWasDigitalInstantiationOf,rico:hasOrHadDerivedInstanciation,rico:hasOrHadPhysicalLocation
0,94,RecordSet,Matrices des propriétés bâties et non bâties,Gentilly,MAT_B_NB_1813,NaN,MatriceCadastrale1807_1822_Bati_NonBati,1813.0,1813.0,1836.0,NaN,NaN,NaN,NaN
1,94,Instanciation,NaN,Gentilly,3P255,NB : 1-542 ; B : 1-166,NaN,NaN,NaN,NaN,MAT_B_NB_1813,NaN,FRAD094_3P_000255_01,Archives Départementales du Val-de-Marne (94)
2,94,Instanciation,NaN,Gentilly,FRAD094_3P_000255_01,NaN,NaN,NaN,NaN,NaN,NaN,MAT_B_NB_1813,NaN,NaN
3,94,RecordSet,Matrice des propriétés foncières (1836-1848),Gentilly,MAT_NB_1836,NaN,MatriceCadastrale1822_1914_NonBati,1836.0,1836.0,1848.0,NaN,NaN,NaN,NaN
4,94,Instanciation,NaN,Gentilly,3P256,1-480,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,FRAD094_3P_000256_01,Archives Départementales du Val-de-Marne (94)
5,94,Instanciation,NaN,Gentilly,3P257,481-980,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,FRAD094_3P_000257_01,Archives Départementales du Val-de-Marne (94)
6,94,Instanciation,NaN,Gentilly,3P258,981-1150,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,FRAD094_3P_000258_01,Archives Départementales du Val-de-Marne (94)
7,94,Instanciation,NaN,Gentilly,FRAD094_3P_000256_01,NaN,NaN,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,NaN
8,94,Instanciation,NaN,Gentilly,FRAD094_3P_000257_01,NaN,NaN,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,NaN
9,94,Instanciation,NaN,Gentilly,FRAD094_3P_000258_01,NaN,NaN,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,NaN


In [8]:
# Print the list of column headers
print(sources.columns.tolist())

['CODE_ARCHIVES', 'rdf:type', 'rico:name', 'rico:location', 'rico:identifier', 'rsfs:comment/Folios', 'hasSourceType', 'hasCreationDate', 'hasValidTime/hasBeginning', 'hasValidTime/hasEnd', 'rico:isOrWasInstantiationOf', 'rico:isOrWasDigitalInstantiationOf', 'rico:hasOrHadDerivedInstanciation', 'rico:hasOrHadPhysicalLocation']


In [9]:
from rdflib import Graph, Literal, Namespace, RDF, URIRef, BNode
from rdflib.namespace import XSD

# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/source/")
g.bind('source', baseuri)

# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
add = Namespace("http://rdf.geohistoricaldata.org/def/address#")
src = Namespace("http://data.ign.fr/def/sources#")
rico = Namespace("https://www.ica.org/standards/RiC/ontology#")
fpo = Namespace("https://github.com/johnBradley501/FPO/raw/master/fpo.owl#")
time = Namespace("http://www.w3.org/2006/time#")
g.bind('cad', cad)
g.bind('add', add)
g.bind('src', src)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

srctype = Namespace("http://data.ign.fr/id/codes/cadastrenap/sourceType/")
mlclasse= Namespace("http://data.ign.fr/id/codes/cadastrenap/mlClasse/")
g.bind('srctype', srctype)
g.bind('mlclasse', mlclasse)

# Iterate over each row in the DataFrame
for index, row in sources.iterrows():

    #URI = http://data.ign.fr/id/sources/{location}_{identifier}
    subject_uri = URIRef(baseuri + f"{str(row['CODE_ARCHIVES'])}_{str(row['rico:location'])}_{str(row['rico:identifier'])}")
    g.add((subject_uri, rico.identifier, Literal(row["rico:identifier"])))

    if row["rdf:type"] == "RecordSet":
        g.add((subject_uri, RDF.type, rico.RecordSet))
        g.add((subject_uri, rico.name, Literal(row["rico:name"])))
        g.add((subject_uri, rico.location, Literal(row["rico:location"])))
        g.add((subject_uri, src.hasSourceType, URIRef(srctype + f'{row["hasSourceType"]}')))
        creationDateBNode = BNode()
        g.add((subject_uri, src.hasCreationDate, creationDateBNode))
        g.add((creationDateBNode, RDF.type, add.CrispTimeInstant))
        g.add((creationDateBNode, add.timeCalendar, time.Gregorian))
        g.add((creationDateBNode, add.timePrecision, time.Year))
        str_date = "{:.0f}".format(row["hasCreationDate"])
        g.add((creationDateBNode, add.timeStamp, Literal(str_date, datatype=XSD.dateTimeStamp)))
        validTimeNode = BNode()
        validTimeNodeBegin = BNode()
        validTimeNodeEnd = BNode()
        g.add((subject_uri, src.hasValidTime, validTimeNode))
        g.add((validTimeNode, RDF.type, time.Interval))
        g.add((validTimeNode, add.hasBeginning, validTimeNodeBegin))
        g.add((validTimeNodeBegin, add.timeCalendar, time.Gregorian))
        g.add((validTimeNodeBegin, add.timePrecision, time.Year))
        str_date = "{:.0f}".format(row["hasValidTime/hasBeginning"])
        g.add((validTimeNodeBegin, add.timeStamp, Literal(str_date, datatype=XSD.dateTimeStamp)))
        g.add((validTimeNode, add.hasEnd, validTimeNodeEnd))
        g.add((validTimeNodeEnd, add.timeCalendar, time.Gregorian))
        g.add((validTimeNodeEnd, add.timePrecision, time.Year))
        str_date = "{:.0f}".format(row["hasValidTime/hasEnd"])
        g.add((validTimeNodeEnd, add.timeStamp, Literal(str_date, datatype=XSD.dateTimeStamp)))
    elif row["rdf:type"] == "Instanciation":
        if row["rico:identifier"][0:2] == '3P' :
            g.add((subject_uri, RDF.type, rico.Instantiation))
            g.add((subject_uri,rico.isOrWasInstantiationOf,URIRef(baseuri + f'{str(row["CODE_ARCHIVES"])}_{row["rico:location"]}_{row["rico:isOrWasInstantiationOf"]}')))
            g.add((subject_uri, rico.hasOrHadDerivedInstanciation, URIRef(baseuri + f'{str(row["CODE_ARCHIVES"])}_{row["rico:location"]}_{row["rico:hasOrHadDerivedInstanciation"]}')))
            g.add((subject_uri,rico.hasOrHadPhysicalLocation,Literal(row['rico:hasOrHadPhysicalLocation'])))
        elif row["rico:identifier"][0:2] != '3P' :
            g.add((subject_uri, RDF.type, rico.Instantiation))
            g.add((subject_uri,rico.isOrWasDigitalInstantiationOf,URIRef(baseuri + f'{str(row["CODE_ARCHIVES"])}_{row["rico:location"]}_{row["rico:isOrWasDigitalInstantiationOf"]}')))
# Print out the graph in Turtle syntax
print(g.serialize(format='turtle'))


@prefix add: <http://rdf.geohistoricaldata.org/def/address#> .
@prefix rico: <https://www.ica.org/standards/RiC/ontology#> .
@prefix source: <http://data.ign.fr/id/source/> .
@prefix src: <http://data.ign.fr/def/sources#> .
@prefix srctype: <http://data.ign.fr/id/codes/cadastrenap/sourceType/> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

source:94_Gentilly_3P255 a rico:Instantiation ;
    rico:hasOrHadDerivedInstanciation source:94_Gentilly_FRAD094_3P_000255_01 ;
    rico:hasOrHadPhysicalLocation "Archives Départementales du Val-de-Marne (94)" ;
    rico:identifier "3P255" ;
    rico:isOrWasInstantiationOf source:94_Gentilly_MAT_B_NB_1813 .

source:94_Gentilly_3P256 a rico:Instantiation ;
    rico:hasOrHadDerivedInstanciation source:94_Gentilly_FRAD094_3P_000256_01 ;
    rico:hasOrHadPhysicalLocation "Archives Départementales du Val-de-Marne (94)" ;
    rico:identifier "3P256" ;
    rico:isOrWasInstantiationOf source:94_Gentilly_M